<a href="https://colab.research.google.com/github/Jane0731/Homework/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9A_W13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 取得股票資訊

## 篩選看漲股票

In [52]:
import yfinance as yf

# 股票清單
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "JNJ", "PFE","NVDA","META","BAC","GS","DIS","NFLX","CVX","XOM"]

# 篩選條件
selected_stocks = []
for ticker in tickers:
    stock = yf.Ticker(ticker)
    pe_ratio = stock.info.get("trailingPE", None)  # 市盈率
    roe = stock.info.get("returnOnEquity", None)  # 股本回報率
    print(pe_ratio)
    print(roe)

    if pe_ratio and roe:
        if pe_ratio < 40 and roe > 0.10:  # 設定條件
            selected_stocks.append(ticker)

print("符合條件的股票:", selected_stocks)


39.837704
1.5741299
36.09076
0.35604
23.125994
0.32101002
45.640167
0.22558
96.21774
0.20389
25.204357
0.20889
34.094593
0.04538
55.18631
1.23767
28.979227
0.36134
16.985348
0.08092
17.635052
0.1017
42.69708
0.0546
51.011196
0.34714
17.788765
0.103520006
14.67009
0.14513999
符合條件的股票: ['AAPL', 'MSFT', 'GOOGL', 'JNJ', 'META', 'GS', 'CVX', 'XOM']


In [53]:
import yfinance as yf
import pandas as pd

# 獲取歷史數據 (過去 1 年的日資料)
stock_data = {}
for ticker in selected_stocks:
    stock = yf.Ticker(ticker)
    data = stock.history(period="5y")
    stock_data[ticker] = data

# 建立 DataFrame 儲存數據
close_prices = pd.DataFrame({ticker: data['Close'] for ticker, data in stock_data.items()})
print(close_prices.columns)
close_prices = close_prices.dropna()  # 移除空值
print("收盤價數據:")
print(close_prices.head())

# 計算每支股票的平均回報與波動率
returns = close_prices.pct_change().mean()  # 平均每日回報
volatility = close_prices.pct_change().std()  # 波動率

# 顯示篩選結果
result = pd.DataFrame({
    "stocks": selected_stocks,
    "return": returns,
    "volatility": volatility
}).sort_values(by="return", ascending=False)

df = pd.DataFrame(result)
df

Index(['AAPL', 'MSFT', 'GOOGL', 'JNJ', 'META', 'GS', 'CVX', 'XOM'], dtype='object')
收盤價數據:
                                AAPL        MSFT      GOOGL         JNJ  \
Date                                                                      
2019-12-05 00:00:00-05:00  64.368782  143.405838  66.184120  121.484383   
2019-12-06 00:00:00-05:00  65.612152  145.146667  66.804085  122.198212   
2019-12-09 00:00:00-05:00  64.693581  144.773636  66.983635  122.302643   
2019-12-10 00:00:00-05:00  65.071678  144.553650  66.978645  121.858704   
2019-12-11 00:00:00-05:00  65.626678  145.098846  67.046486  122.720459   

                                 META          GS        CVX        XOM  
Date                                                                     
2019-12-05 00:00:00-05:00  198.761551  192.128113  93.141861  53.677414  
2019-12-06 00:00:00-05:00  200.446487  198.737640  94.486969  54.540516  
2019-12-09 00:00:00-05:00  200.735611  196.260162  93.918488  54.658222  
2019-12-10 00

,stocks,return,volatility
META,META,0.001302,0.028261
AAPL,AAPL,0.001256,0.019960
GS,GS,0.001121,0.020831
MSFT,MSFT,0.001071,0.019176
GOOGL,GOOGL,0.000977,0.020292
XOM,XOM,0.000836,0.021658
CVX,CVX,0.000675,0.022375
JNJ,JNJ,0.000246,0.012350


In [64]:
import numpy as np
import random

# 初始化參數
population_size = 100
num_generations = 500
mutation_rate = 0.2
chromosome_length = len(df["stocks"])

# 適應函數 (目標是最大化回報，最小化風險)
def fitness_function(chromosome):
    total_return = np.sum(np.array(df["return"]) * chromosome)
    total_risk = np.sum(np.array(df["volatility"]) * chromosome)
    num_selected_stocks = np.sum(chromosome)
    if total_risk == 0 or num_selected_stocks > 6 or num_selected_stocks < 3:
        return 0
    diversity_bonus = num_selected_stocks * 0.01  # 每多選一支股票增加適應值
    return total_return / (total_risk + 1e-6) + diversity_bonus





# 初始化族群
def initialize_population():
    population = []
    for _ in range(population_size):
        chromosome = np.random.randint(0, 2, chromosome_length)
        # 控制初始選股數量
        while np.sum(chromosome) > 9 or np.sum(chromosome) < 5:
            chromosome = np.random.randint(0, 2, chromosome_length)
        # 強制選擇高回報股票
        top_indices = np.argsort(-np.array(df["return"]))[:5]
        chromosome[top_indices] = 1
        population.append(chromosome)
    return population




# 選擇 (輪盤選擇)
def selection(population, fitness_scores):
    # 避免負數值
    fitness_scores = np.maximum(fitness_scores, 0)  # 將負數值設置為 0
    total_fitness = sum(fitness_scores)
    if total_fitness == 0:
        # 如果所有 fitness_scores 都是 0，隨機選擇
        return random.choice(population)  # 修正為使用 random.choice
    probabilities = [f / total_fitness for f in fitness_scores]
    selected_index = np.random.choice(range(len(population)), p=probabilities)
    return population[selected_index]  # 返回選擇的染色體


# 交配
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, chromosome_length - 1)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

# 突變
def mutation(chromosome):
    for i in range(chromosome_length):
        if np.random.rand() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

# 修改精英保留策略
def genetic_algorithm():
    population = initialize_population()
    best_solution = None
    best_fitness = -float("inf")

    for generation in range(num_generations):
        fitness_scores = [fitness_function(chromosome) for chromosome in population]

        # 更新最佳解
        for i, fitness in enumerate(fitness_scores):
            if fitness > best_fitness:
                best_fitness = fitness
                best_solution = population[i]

        elite_count = max(1, population_size // 10)  # 保留前 20% 的精英個體
        elite_indices = np.argsort(fitness_scores)[-elite_count:]
        elites = [population[i] for i in elite_indices]


        # 新的族群
        new_population = []
        for _ in range((population_size - elite_count) // 2):
            parent1 = selection(population, fitness_scores)
            parent2 = selection(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutation(child1), mutation(child2)])

        # 加入精英個體
        new_population.extend(elites)
        population = new_population

    return best_solution, best_fitness


# 執行演算法
best_solution, best_fitness = genetic_algorithm()

# 輸出結果
selected_stocks = [df["stocks"][i] for i in range(chromosome_length) if best_solution[i] == 1]
total_return = np.sum(np.array(df["return"]) * best_solution)
total_risk = np.sum(np.array(df["volatility"]) * best_solution)

print("最佳投資組合:", selected_stocks)
print("總回報率:", total_return)
print("總風險:", total_risk)
print("適應函數值:", best_fitness)

最佳投資組合: ['META', 'AAPL', 'GS', 'MSFT', 'GOOGL', 'XOM']
總回報率: 0.006562975611301576
總風險: 0.13017709300107455
適應函數值: 0.11041536144831529


<ipython-input-64-9aab457f21d0>:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  selected_stocks = [df["stocks"][i] for i in range(chromosome_length) if best_solution[i] == 1]
